In [ ]:
# import data 
import pandas as pd
S1 = pd.read_excel('pancancer1.xlsx')
cosmic = pd.read_excel('COSMIC.xlsx')

In [20]:
# merge dataframes
cosmic["Ensembl gene id"] = cosmic.Ensembl_gene_ID
complete_data = cosmic.join(S1.set_index("Ensembl gene id"), "Ensembl gene id", how="left")
complete_data = complete_data.groupby("Ensembl gene id").first().reset_index()
#clean up data and creating working dataframe 
delete = ['Ensembl gene id', 'Chromosome', 'Start_position', 'Variant id', 'End_position', 'Annotation_sites', 'Uniprots id', 'Gene symbol']
working_data = complete_data.drop(columns = delete) 

In [25]:
# create a numerical index for classifying base pair substitutions
bp_sub = [0] * working_data.shape[0]
# A, T, G, C
#for i, r, in working_data.head(50).iterrows():
    #print( r["Reference_nucleotide"] ,r['Altered_nucleotide'])
for i,ref in enumerate(working_data['Reference_nucleotide']):
    if str(ref) == 'A' and str(working_data.loc[i, 'Altered_nucleotide']) == 'A':
        bp_sub[i] = 0
    elif str(ref) == 'A' and str(working_data.loc[i, 'Altered_nucleotide']) == 'T':
        bp_sub[i] = 1
    elif str(ref) == 'A' and str(working_data.loc[i, 'Altered_nucleotide']) == 'G':
        bp_sub[i] = 2
    elif str(ref) == 'A' and str(working_data.loc[i, 'Altered_nucleotide']) == 'C':
        bp_sub[i] = 3
    elif str(ref) == 'T' and str(working_data.loc[i, 'Altered_nucleotide']) == 'A':
        bp_sub[i] = 4
    elif str(ref) == 'T' and str(working_data.loc[i, 'Altered_nucleotide']) == 'T':
        bp_sub[i] = 0
    elif str(ref) == 'T' and str(working_data.loc[i, 'Altered_nucleotide']) == 'G':
        bp_sub[i] = 5
    elif str(ref) == 'T' and str(working_data.loc[i, 'Altered_nucleotide']) == 'C':
        bp_sub[i] = 6
    elif str(ref) == 'G' and str(working_data.loc[i, 'Altered_nucleotide']) == 'A':
        bp_sub[i] = 7
    elif str(ref) == 'G' and str(working_data.loc[i, 'Altered_nucleotide']) == 'T':
        bp_sub[i] = 8
    elif str(ref) == 'G' and str(working_data.loc[i, 'Altered_nucleotide']) == 'G':
        bp_sub[i] = 0
    elif str(ref) == 'G' and str(working_data.loc[i, 'Altered_nucleotide']) == 'C':
        bp_sub[i] = 9
    elif str(ref) == 'C' and str(working_data.loc[i, 'Altered_nucleotide']) == 'A':
        bp_sub[i] = 10
    elif str(ref) == 'C' and str(working_data.loc[i, 'Altered_nucleotide']) == 'T':
        bp_sub[i] = 11
    elif str(ref) == 'C' and str(working_data.loc[i, 'Altered_nucleotide']) == 'G':
        bp_sub[i] = 12
    elif str(ref) == 'C' and str(working_data.loc[i, 'Altered_nucleotide']) == 'C':
        bp_sub[i] = 0
working_data['Base Pair Substitutions'] = bp_sub

In [26]:
working_data.head(5)

,Reference_nucleotide,Altered_nucleotide,Ensembl_gene_ID,Amino_acid_substitution,PON-P2_predicted_probability,PON-P2_error,PON-P2_classification,Frequency_in_COSMIC,Neoantigen freq,Variant freq,Protein length,Length-normalized variant freq,Base Pair Substitutions
0,C,T,ENSG00000000457,R61C,0.950,0.022,Pathogenic,6,919.0,53.0,742.0,0.071429,11
1,G,A,ENSG00000000460,V383I,0.009,0.006,Neutral,3,NaN,NaN,NaN,NaN,7
2,A,G,ENSG00000000938,R26G,0.493,0.064,Unknown,1,492.0,56.0,529.0,0.105860,2
3,G,A,ENSG00000000971,V72I,0.077,0.025,Neutral,3,3043.0,76.0,1231.0,0.061738,7
4,A,G,ENSG00000001036,R159G,0.278,0.112,Unknown,3,664.0,132.0,467.0,0.282655,2


In [ ]:
# create a numberical index for pathogenicity classification
pathogenicity_classification = []
for pathogenicity in working_data['PON-P2_classification']:
    if str(pathogenicity) == 'Pathogenic':
        pathogenicity_classification.append(1)
    else:
        pathogenicity_classification.append(0)
working_data['Pathogenicity Classification'] = pathogenicity_classification

In [ ]:
aa_classification = []
np_neutral = ['W', 'F', 'G', 'A', 'V', 'I', 'L', 'M', 'P']
p_neutral = ['Y', 'S', 'T', 'N', 'Q', 'C']
p_acidic = ['D', 'E']
p_basic = ['R', 'K', 'H']
for aminoacid in working_data['Amino_acid_substitution']:
    wt_aa = str(aminoacid)[0]
    mut_aa = str(aminoacid)[-1]
    if wt_aa in np_neutral and mut_aa in p_neutral:
        aa_classification.append(1)
    elif wt_aa in np_neutral and mut_aa in p_acidic:
        aa_classification.append(2)
    elif wt_aa in np_neutral and mut_aa in p_basic:
        aa_classification.append(3)
    elif wt_aa in p_neutral and mut_aa in np_neutral:
        aa_classification.append(4)
    elif wt_aa in p_neutral and mut_aa in p_acidic:
        aa_classification.append(5)
    elif wt_aa in p_neutral and mut_aa in p_basic:
        aa_classification.append(6)
    elif wt_aa in p_acidic and mut_aa in np_neutral:
        aa_classification.append(7)
    elif wt_aa in p_acidic and mut_aa in p_neutral:
        aa_classification.append(8)
    elif wt_aa in p_acidic and mut_aa in p_basic:
        aa_classification.append(9)
    elif wt_aa in p_basic and mut_aa in np_neutral:
        aa_classification.append(10)
    elif wt_aa in p_basic and mut_aa in p_neutral:
        aa_classification.append(11)
    elif wt_aa in p_basic and mut_aa in p_acidic:
        aa_classification.append(12)
    else:
        aa_classification.append(0)
working_data['Amino Acid Classification'] = aa_classification

In [11]:
# Base-Pair Sub Key
import pandas as pd
key = pd.DataFrame(data = ['None','A to T', 'A to G', 'A to C', 'T to A', 'T to G', 'T to C', 'G to A', 'G to T', 'G to C', 'C to A', 'C to T', 'C to G'], columns = ['Base Pair Substitution'], index  = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])
key['Pathogenicity'] = ['Pathogenic', 'Neutral/Unknown', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A']
key['Amino Acid Sub'] = ['None', 'NP neutral to P neutral', 'NP neutral to P acidic', 'NP neutral to P basic', 'P neutral to NP neutral', 'P neutral to P acidic', 'P neutral to P basic', 'P acidic to NP neutral', 'P acidic to P neutral', 'P acidic to P basic', 'P basic to NP neutral', 'P basic to P neutral', 'P basic to P acidic']

In [ ]:
numerical_data = working_data.drop(columns = ['Reference_nucleotide', 'Altered_nucleotide', 'PON-P2_classification'])
working_data.sort_values(by = 'Ensembl_gene_ID', ascending = False)